In [1]:
import pandas as pd
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain_community.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate
from langchain_community.vectorstores import chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from langchain import HuggingFacePipeline
from transformers import pipeline
# RAG Chain
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import RetrievalQA
from transformers import AutoModelForCausalLM,AutoTokenizer
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from langchain import HuggingFacePipeline
import transformers
import torch  
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.chat_models import ChatOllama
from dotenv import load_dotenv


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Defining the model

In [2]:
llm = ChatOllama(
            temperature=1, 
            model="llama3.1:8b",
            max_tokens=600,
            top_p=0.90,
        #     frequency_penalty=1,
        #     presence_penalty=1,
    )

C:\Users\HP\AppData\Local\Temp\ipykernel_25476\1625332078.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(


In [3]:
llm = ChatOllama(model="llama3.1:8b", temperature=0.5)
llm.invoke("Warm up") 

AIMessage(content="Here are a few options to get us started:\n\n1. **Word association game**: I'll start with a word, and you respond with a word that's associated with it.\n2. **20 Questions**: You think of an object, and I try to guess what it is by asking yes or no questions.\n3. **Hangman**: I'll think of a word or phrase, and you try to guess it by guessing letters.\n4. **Trivia**: We can play a quick trivia game on a topic of your choice (e.g., history, science, sports, music).\n5. **Word scrambles**: I'll scramble a word, and you try to unscramble it.\n\nWhich one would you like to do? Or feel free to suggest something else!", additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-09-22T13:44:11.5409089Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 8581366400, 'load_duration': 75631000, 'prompt_eval_count': 12, 'prompt_eval_duration': 312913600, 'eval_count': 158, 'eval_duration': 81910272

In [4]:
# model = 'microsoft/phi-2'

# tokenizer = AutoTokenizer.from_pretrained(model)

# model = AutoModelForCausalLM.from_pretrained(model,torch_dtype="auto",device_map='auto')

# pipe = pipeline("text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     device_map="auto",
#     max_new_tokens=256,
#     temperature=0.5)

#### Embbeding and using vector database

In [5]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="multi-qa-MiniLM-L6-cos-v1")
persist_directory = "./chroma_capstone_db_new_reduced_hugging_face"

# Try accessing known collection
collection_name = "Tomato"  # Replace with any collection you expect
chroma_db = Chroma(
    collection_name=collection_name,
    persist_directory=persist_directory,
    embedding_function=embedding
)

print(f"Collection {collection_name} loaded. Number of documents:", chroma_db._collection.count())

C:\Users\HP\AppData\Local\Temp\ipykernel_25476\2021234551.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="multi-qa-MiniLM-L6-cos-v1")


Collection Tomato loaded. Number of documents: 4207


### Chromadb as Retriever using mmr

In [6]:
question =  "give me the cure for Tomato plant for Aphids disease"
chroma_retriever = chroma_db.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 6,
        "fetch_k": 12,
        "filter": {
            "$and": [
                {"StateName": {"$eq": "ASSAM"}},
                {"disease_name": {"$eq": "Aphids"}}
            ]
        }
    }
)

# chroma_retriever = chroma_db.as_retriever(
#     search_type="mmr",
#     search_kwargs={
#         "k": 6,
#         "fetch_k": 12,
#         # "filter": {
#         #     "$and": [
#         #         {"StateName": {"$eq": "ASSAM"}},
#         #         {"disease_name": {"$eq": "Aphids"}}
#         #     ]
#         # }
#     }
# )


chroma_retriever.get_relevant_documents(question)

C:\Users\HP\AppData\Local\Temp\ipykernel_25476\1855896669.py:31: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chroma_retriever.get_relevant_documents(question)


[Document(id='e49e4c69-b0f3-4757-a4ac-8a42339ecdab', metadata={'StateName': 'ASSAM', 'disease_name': 'Aphids'}, page_content='Crop:Tomato\nDisease:Aphids\nStateName:ASSAM\nSeason_English:nan\nMonth:January\nQueryText:ASKING ABOUT CONTROL OF APHIDS\nKccAns:APPLY  ROGOR   1-2MLLIT OF WATER SPRAY 3 TIMES AT 5 DAYS INTERVALFOR CONTROL OF LATE BLIGHT APPLY RIDOMIL GOLD 2GLIT OF WATER SPRAY 3 TIME AT 5 DAYS INTERVAL'),
 Document(id='fa88fa51-c230-4bff-8a97-c258c1f07ebc', metadata={'disease_name': 'Aphids', 'StateName': 'ASSAM'}, page_content='Crop:Tomato\nDisease:Aphids\nStateName:ASSAM\nSeason_English:nan\nMonth:February\nQueryText:Query regarding plant protection of tomato against aphids\nKccAns:If infestation is low apply AZADIRACHTIN NEEM KAVACH NEEMOL NEEMEX 2mlL OF WATER If infestation is high apply IMIDACLOPRID  COFIDORMIDAS 2000 2mlL OF WATER 7-10 DAYS INTERVAL'),
 Document(id='6552196f-9000-413b-a1d8-b763addc134e', metadata={'StateName': 'ASSAM', 'disease_name': 'Aphids'}, page_cont

#### Prompting

In [7]:
prompt_template = """
You are an agricultural assistant specialized in providing answers about plant diseases.

Instructions:
1. Use the provided CONTEXT from the documents to answer the QUESTION. This context contains official guidance for the specific crop.
2. The CONTEXT may include: DistrictName, StateName, Season_English, Month, Disease, QueryText, and KccAns (official answer).
3. Always prioritize KccAns when it is available. Do not invent or assume details if KccAns provides the answer.
4. Use DistrictName, StateName, Season_English, Month, and Disease only to understand and interpret the question. Do not include them in your answer unless explicitly requested.
5. If the CONTEXT has no relevant information, you may provide general agricultural advice.

CONTENTS FROM RAG:
{context}

QUESTION:
{question}

ANSWER:
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

#### Retriever

In [8]:
h_retrieval_QA1 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=chroma_retriever,
    input_key="query",
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [9]:
output = h_retrieval_QA1.invoke({"query": question})

In [11]:
result = output["result"]

In [12]:
result

'Since there are multiple entries related to Tomato and Aphids, I will provide the most relevant and specific answer based on the available KccAns.\n\nFor Tomato plants affected by Aphids in ASSAM during different months:\n\n* In January: No specific guidance is provided for Aphids control.\n* In February: If infestation is low, apply AZADIRACHTIN NEEM KAVACH NEEMOL NEEMEX 2mlL of water. If infestation is high, apply IMIDACLOPRID COFIDORMIDAS 2000 2mlL of water at 7-10 days interval.\n* In March: Apply THIOMETHOXAM or IMIDACLOPRID 2GML water against Aphids and also soil drenching with BLITOX50 2GML water against Damping Off (not directly related to Aphids, but mentioned in the same entry).\n* In April: Suggested to apply ROGOR 35EC 2ML per litre of water.\n* In November (Monsoon Crop): Apply CAPTAN or BAVISTIN one tea spoon 2 LITER OF WATER for fungal wilt control and ROGOR 35 EC 1 MLLITER OF WATER for Aphids control.\n\nSince the question does not specify a particular month, I will pr

In [14]:
output["source_documents"]

[Document(id='e49e4c69-b0f3-4757-a4ac-8a42339ecdab', metadata={'disease_name': 'Aphids', 'StateName': 'ASSAM'}, page_content='Crop:Tomato\nDisease:Aphids\nStateName:ASSAM\nSeason_English:nan\nMonth:January\nQueryText:ASKING ABOUT CONTROL OF APHIDS\nKccAns:APPLY  ROGOR   1-2MLLIT OF WATER SPRAY 3 TIMES AT 5 DAYS INTERVALFOR CONTROL OF LATE BLIGHT APPLY RIDOMIL GOLD 2GLIT OF WATER SPRAY 3 TIME AT 5 DAYS INTERVAL'),
 Document(id='fa88fa51-c230-4bff-8a97-c258c1f07ebc', metadata={'disease_name': 'Aphids', 'StateName': 'ASSAM'}, page_content='Crop:Tomato\nDisease:Aphids\nStateName:ASSAM\nSeason_English:nan\nMonth:February\nQueryText:Query regarding plant protection of tomato against aphids\nKccAns:If infestation is low apply AZADIRACHTIN NEEM KAVACH NEEMOL NEEMEX 2mlL OF WATER If infestation is high apply IMIDACLOPRID  COFIDORMIDAS 2000 2mlL OF WATER 7-10 DAYS INTERVAL'),
 Document(id='6552196f-9000-413b-a1d8-b763addc134e', metadata={'disease_name': 'Aphids', 'StateName': 'ASSAM'}, page_cont

In [ ]:
context = [output.page_content for output in output["source_documents"]]

In [23]:
context

['Crop:Tomato\nDisease:Aphids\nStateName:ASSAM\nSeason_English:nan\nMonth:January\nQueryText:ASKING ABOUT CONTROL OF APHIDS\nKccAns:APPLY  ROGOR   1-2MLLIT OF WATER SPRAY 3 TIMES AT 5 DAYS INTERVALFOR CONTROL OF LATE BLIGHT APPLY RIDOMIL GOLD 2GLIT OF WATER SPRAY 3 TIME AT 5 DAYS INTERVAL',
 'Crop:Tomato\nDisease:Aphids\nStateName:ASSAM\nSeason_English:nan\nMonth:February\nQueryText:Query regarding plant protection of tomato against aphids\nKccAns:If infestation is low apply AZADIRACHTIN NEEM KAVACH NEEMOL NEEMEX 2mlL OF WATER If infestation is high apply IMIDACLOPRID  COFIDORMIDAS 2000 2mlL OF WATER 7-10 DAYS INTERVAL',
 'Crop:Tomato\nDisease:Aphids\nStateName:ASSAM\nSeason_English:nan\nMonth:April\nQueryText:ASKING ABOUT THE CONTROL OF APHIDS IN TOMATO\nKccAns:SUGGESTED TO APPLY THE ROGOR 35EC 2ML PER LITRE OF WATER',
 'Crop:Tomato\nDisease:Aphids\nStateName:ASSAM\nSeason_English:nan\nMonth:March\nQueryText:ASKING ABOUT THE CONTROL OF APHIDS\nKccAns:THIOMETHOXAM OR IMIDACLOPRID  2GML

In [15]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [16]:
evaluator_llm = ChatOpenAI(
    model_name="gpt-4",   # or "gpt-4-turbo"
    temperature=0,         # deterministic output
    openai_api_key=os.environ["OPENAI_API_KEY"]  # optional if set as env variable
)

C:\Users\HP\AppData\Local\Temp\ipykernel_25476\819834627.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  evaluator_llm = ChatOpenAI(


In [24]:
from ragas.dataset_schema import SingleTurnSample 
from ragas.metrics import Faithfulness


sample = SingleTurnSample(
        user_input=question,
        response=result,
        retrieved_contexts= context
    )
scorer = Faithfulness(llm=evaluator_llm)
await scorer.single_turn_ascore(sample)

0.7333333333333333

In [25]:
from ragas import SingleTurnSample 
from ragas.metrics import ResponseRelevancy

embedding_model = HuggingFaceEmbeddings(model_name="multi-qa-MiniLM-L6-cos-v1")

sample = SingleTurnSample(
        user_input=question,
        response=result,
        retrieved_contexts=context
    )

scorer = ResponseRelevancy(llm=evaluator_llm, embeddings=embedding_model)
await scorer.single_turn_ascore(sample)

np.float64(0.8215532579912604)

In [26]:
from ragas.dataset_schema import SingleTurnSample
from ragas.metrics import AspectCritic

sample = SingleTurnSample(
    user_input=question,
    response=result,
)

scorer =  AspectCritic(
        name="maliciousness",
        definition="Is the submission intended to harm, deceive, or exploit users?",
        llm=evaluator_llm

    )
await scorer.single_turn_ascore(sample)

0

In [29]:
from ragas import SingleTurnSample
from ragas.metrics import LLMContextPrecisionWithoutReference

context_precision = LLMContextPrecisionWithoutReference(llm=evaluator_llm)

sample = SingleTurnSample(
    user_input=question,
    response=result,
    retrieved_contexts=context, 
)


await context_precision.single_turn_ascore(sample)

0.9999999999833333